<a href="https://colab.research.google.com/github/Spycsh/DataScienceNoteBooks/blob/main/rdkit_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install RDKit. Takes 2-3 minutes
!wget -c https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!time bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

--2020-12-01 17:16:33--  https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh [following]
--2020-12-01 17:16:33--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88867207 (85M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.3-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  84.75M   150MB/s    in 0.6s    

2020-12-01 17:16:34 (150 MB/s) - ‘Minicon

In [4]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [6]:
import numpy as np
import pandas as pd

In [5]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole

m = Chem.MolFromSmiles('[O-][Cl+3]([O-])([O-])[O-].c1ccc(-c2[nH]c[n+](CC3CO3)c2-c2ccccc2)cc1')
print(m.GetNumAtoms())

26


In [55]:
# https://www.rdkit.org/docs/source/rdkit.Chem.rdMolDescriptors.html
# compute molecular descriptors
import rdkit.Chem.rdMolDescriptors as d
print(d.CalcExactMolWt(m))  # returns the molecule’s exact molecular weight
# https://www.rdkit.org/docs/source/rdkit.Chem.Fragments.html
# functions to match a bunch of fragment descriptors from a file
import rdkit.Chem.Fragments as f
print(f.fr_COO(m)) # Number of aliphatic carboxylic acids
# https://www.rdkit.org/docs/source/rdkit.Chem.Lipinski.html
# Calculation of Lipinski parameters for molecules
import rdkit.Chem.Lipinski as l
print(l.HeavyAtomCount(m)) # Number of heavy atoms (any atom that is not hydrogen) a molecule.
# A special class of features is the so-called fingerprints, which represent presence or absence of substructures. They can be derived in many different ways. 
# One of these that is included in RDKit is the so-called Morgan fingerprints, 
# Here, the second argument corresponds to the size of the substructures
# and the third argument corresponds to how many dimensions to map the 
# substructures to (length of the bit vector)
# which can be generated as follows:
from rdkit.Chem import AllChem
fp = AllChem.GetMorganFingerprintAsBitVect(m,2,nBits=124)
print(np.array(fp))

376.08259932400006
0
26
[0 0 1 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0
 0 0 1 1 0 1 0 1 0 0 0 1 0]


The task should include

1. what alternative representations (choice of feature sets and data preparation techniques) and learning algorithms (including parameter settings) have been evaluated (at least two different representations and two different algorithms have to be evaluated), and possibly also what approaches to combine multiple models and feature sets have been evaluated. Provide pointers to all external packages that you have used.

2. what method has been employed to choose model and what model has been chosen, what method has been used to provide an estimate for the AUC and what is the estimate for the chosen model.

In [9]:
# read the dataset 
train_df = pd.read_csv("training_smiles.csv")
train_df.head(5)

,INDEX,SMILES,ACTIVE
0,1,CC#CCCCC(=O)Nc1ccccc1C(=O)O,0.0
1,2,[O-][Cl+3]([O-])([O-])[O-].c1ccc(-c2[nH]c[n+](...,0.0
2,3,CCOC(=O)CSc1nnc(NC(=O)c2cccc([N+](=O)[O-])c2C)s1,0.0
3,4,O=C(CN1CCN(S(=O)(=O)c2ccccc2)CC1)Nc1ccc(Cl)c(C...,0.0
4,5,Cc1cc(NN/C=C2\C=CC(=O)C=C2O)nc(N2CCOCC2)n1,0.0


**Extract features**

* CalcExactMolWt
* HeavyAtomCount
* fr_COO
* MFbitV_x

potential features
* number of atoms
* molecule weights
* fragments
* Lipinski parameters
* Morgan fingerpints(substructures)

## Feature extracting

For quick start, just skip this section and read the 
train_new_features.csv (which has already saved after execeuting
this section)

In [ ]:
train_df["HeavyAtomCount"] = train_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x).GetNumAtoms())
train_df["CalcExactMolWt"] = train_df["SMILES"].apply(lambda x: d.CalcExactMolWt(Chem.MolFromSmiles(x)))
train_df["fr_COO"] = train_df["SMILES"].apply(lambda x: f.fr_Al_COO(Chem.MolFromSmiles(x)))

RDKit WARNING: [20:20:54] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [20:21:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [20:21:29] WARNING: not removing hydrogen atom without neighbors


In [ ]:
train_df

,INDEX,SMILES,ACTIVE,HeavyAtomCount,CalcExactMolWt,fr_COO,MFbitV_0,MFbitV_1,MFbitV_2,MFbitV_3,MFbitV_4,MFbitV_5,MFbitV_6,MFbitV_7,MFbitV_8,MFbitV_9,MFbitV_10,MFbitV_11,MFbitV_12,MFbitV_13,MFbitV_14,MFbitV_15,MFbitV_16,MFbitV_17,MFbitV_18,MFbitV_19,MFbitV_20,MFbitV_21,MFbitV_22,MFbitV_23,MFbitV_24,MFbitV_25,MFbitV_26,MFbitV_27,MFbitV_28,MFbitV_29,MFbitV_30,MFbitV_31,MFbitV_32,MFbitV_33,...,MFbitV_84,MFbitV_85,MFbitV_86,MFbitV_87,MFbitV_88,MFbitV_89,MFbitV_90,MFbitV_91,MFbitV_92,MFbitV_93,MFbitV_94,MFbitV_95,MFbitV_96,MFbitV_97,MFbitV_98,MFbitV_99,MFbitV_100,MFbitV_101,MFbitV_102,MFbitV_103,MFbitV_104,MFbitV_105,MFbitV_106,MFbitV_107,MFbitV_108,MFbitV_109,MFbitV_110,MFbitV_111,MFbitV_112,MFbitV_113,MFbitV_114,MFbitV_115,MFbitV_116,MFbitV_117,MFbitV_118,MFbitV_119,MFbitV_120,MFbitV_121,MFbitV_122,MFbitV_123
0,1,CC#CCCCC(=O)Nc1ccccc1C(=O)O,0.0,18,245.105193,0,0,0,0,0,0,1,1,1,1,0,1,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0
1,2,[O-][Cl+3]([O-])([O-])[O-].c1ccc(-c2[nH]c[n+](...,0.0,26,376.082599,0,0,0,1,1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0
2,3,CCOC(=O)CSc1nnc(NC(=O)c2cccc([N+](=O)[O-])c2C)s1,0.0,25,382.040562,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,1,0,1,1,0,0,...,1,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,1,1,0,1,0
3,4,O=C(CN1CCN(S(=O)(=O)c2ccccc2)CC1)Nc1ccc(Cl)c(C...,0.0,27,427.052418,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0
4,5,Cc1cc(NN/C=C2\C=CC(=O)C=C2O)nc(N2CCOCC2)n1,0.0,24,329.148789,0,0,0,1,1,0,0,0,0,0,1,1,0,1,0,0,1,0,1,1,1,0,0,0,1,0,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,1,1,1,0,1,0,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121369,121370,O=C(NCc1cccs1)C1CCCN(S(=O)(=O)c2cnc[nH]2)C1,0.0,23,354.082032,0,0,0,1,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,1,0,0,...,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,1,0,0,1,0
121370,121371,COc1ccc(Cn2nc(C)cc2C)cc1OC,0.0,18,246.136828,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0
121371,121372,Cc1ccc(-c2nn(-c3cc(Cl)ccc3[N+](=O)[O-])c(=O)c3...,0.0,29,405.088019,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,...,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,1,1,0,0,0,0,0,0,1,1,1,0
121372,121373,O=C(OCCN1C(=O)c2ccccc2C1=O)c1cccc(OC(F)F)c1,0.0,26,361.076179,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,1,0,1,1,0,0,0,0,0,1,0,1,0,...,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,0,0,1,0


In [ ]:
## just for test
# kk = train_df.copy()
# for i in range(124):
#   kk["MFbitV_"+str(i)] = 0
# # kk.iloc[:5, 6:] 5*124
# x = train_df["SMILES"].iloc[:5].apply(lambda x: AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(x),2,nBits=124))
# # x[0].tolist()
# lists = []
# for i in x:
#   lists += np.array(i).tolist()
# # lists
# # pd.DataFrame((np.array(lists).reshape(5,124)))
# kk.iloc[:5,6:] = np.array(lists).reshape(5,124)
# kk

In [ ]:
for i in range(124):
  train_df["MFbitV_"+str(i)] = 0

print("wait 5 minutes please...")
# 121374*124 MF vector
MF_x = train_df["SMILES"].apply(lambda x: np.array(AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(x),2,nBits=124)))

# vector_lists = []
# # concat the lists
# for i in MF_x:
#   vector_lists += np.array(i).tolist()
import itertools
vector_lists = list(itertools.chain(*MF_x))

RDKit WARNING: [20:29:37] WARNING: not removing hydrogen atom without neighbors


wait 5 minutes please...


In [ ]:
train_df.iloc[:,6:] = np.array(vector_lists).reshape(len(train_df),124)

In [ ]:
train_df.to_csv('train_new_features.csv', index=False)

skip the above section, directly read the training_preparation.csv in which features have already been
extracted.

In [10]:
train_df = pd.read_csv('train_new_features.csv')

## Data preprocessing
* column filter
* imputation
* normalization
* discretization
* train test split

In [13]:
from sklearn import model_selection
df_X = train_df.drop(columns=['INDEX','SMILES','ACTIVE'])
y = train_df['ACTIVE']

Normalization across instances should be done after splitting the data between training and test set, using only the data from the training set.

This is because the test set plays the role of fresh unseen data, so it's not supposed to be accessible at the training stage. Using any information coming from the test set before or during training is a potential bias in the evaluation of the performance

In [14]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    df_X, y, test_size=0.2, random_state=1) # set the random state
y_train.head()

27745    0.0
4208     0.0
22527    0.0
7579     0.0
32382    0.0
Name: ACTIVE, dtype: float64

In [15]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from sklearn.preprocessing import KBinsDiscretizer
kbd = KBinsDiscretizer(n_bins=10, encode="ordinal")

In [16]:
# save the indexes
train_indexes = X_train.index
test_indexes = X_test.index

In [17]:
X_train = imp_mean.fit_transform(X_train)
X_train[:,[0,1]] = scaler.fit_transform(X_train[:,[0,1]])
X_train[:,[0,1]] = kbd.fit_transform(X_train[:,[0,1]])
X_train = pd.DataFrame(X_train, columns=df_X.columns, index=train_indexes)
X_train
# df_X = scaler.fit_transform(X_train)
# df_X = kbd.fit_transform(df_X)
# pd.DataFrame(df_X)

,HeavyAtomCount,CalcExactMolWt,fr_COO,MFbitV_0,MFbitV_1,MFbitV_2,MFbitV_3,MFbitV_4,MFbitV_5,MFbitV_6,MFbitV_7,MFbitV_8,MFbitV_9,MFbitV_10,MFbitV_11,MFbitV_12,MFbitV_13,MFbitV_14,MFbitV_15,MFbitV_16,MFbitV_17,MFbitV_18,MFbitV_19,MFbitV_20,MFbitV_21,MFbitV_22,MFbitV_23,MFbitV_24,MFbitV_25,MFbitV_26,MFbitV_27,MFbitV_28,MFbitV_29,MFbitV_30,MFbitV_31,MFbitV_32,MFbitV_33,MFbitV_34,MFbitV_35,MFbitV_36,...,MFbitV_84,MFbitV_85,MFbitV_86,MFbitV_87,MFbitV_88,MFbitV_89,MFbitV_90,MFbitV_91,MFbitV_92,MFbitV_93,MFbitV_94,MFbitV_95,MFbitV_96,MFbitV_97,MFbitV_98,MFbitV_99,MFbitV_100,MFbitV_101,MFbitV_102,MFbitV_103,MFbitV_104,MFbitV_105,MFbitV_106,MFbitV_107,MFbitV_108,MFbitV_109,MFbitV_110,MFbitV_111,MFbitV_112,MFbitV_113,MFbitV_114,MFbitV_115,MFbitV_116,MFbitV_117,MFbitV_118,MFbitV_119,MFbitV_120,MFbitV_121,MFbitV_122,MFbitV_123
27745,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
4208,2.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
22527,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
7579,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
32382,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50057,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
98047,4.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
5192,5.0,5.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
77708,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0

In [18]:
X_test = imp_mean.fit_transform(X_test)
X_test[:,[0,1]] = scaler.fit_transform(X_test[:,[0,1]])
X_test[:,[0,1]] = kbd.fit_transform(X_test[:,[0,1]])
X_test = pd.DataFrame(X_test, columns=df_X.columns, index=test_indexes)
X_test

,HeavyAtomCount,CalcExactMolWt,fr_COO,MFbitV_0,MFbitV_1,MFbitV_2,MFbitV_3,MFbitV_4,MFbitV_5,MFbitV_6,MFbitV_7,MFbitV_8,MFbitV_9,MFbitV_10,MFbitV_11,MFbitV_12,MFbitV_13,MFbitV_14,MFbitV_15,MFbitV_16,MFbitV_17,MFbitV_18,MFbitV_19,MFbitV_20,MFbitV_21,MFbitV_22,MFbitV_23,MFbitV_24,MFbitV_25,MFbitV_26,MFbitV_27,MFbitV_28,MFbitV_29,MFbitV_30,MFbitV_31,MFbitV_32,MFbitV_33,MFbitV_34,MFbitV_35,MFbitV_36,...,MFbitV_84,MFbitV_85,MFbitV_86,MFbitV_87,MFbitV_88,MFbitV_89,MFbitV_90,MFbitV_91,MFbitV_92,MFbitV_93,MFbitV_94,MFbitV_95,MFbitV_96,MFbitV_97,MFbitV_98,MFbitV_99,MFbitV_100,MFbitV_101,MFbitV_102,MFbitV_103,MFbitV_104,MFbitV_105,MFbitV_106,MFbitV_107,MFbitV_108,MFbitV_109,MFbitV_110,MFbitV_111,MFbitV_112,MFbitV_113,MFbitV_114,MFbitV_115,MFbitV_116,MFbitV_117,MFbitV_118,MFbitV_119,MFbitV_120,MFbitV_121,MFbitV_122,MFbitV_123
31170,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
59201,8.0,8.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
74232,5.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
65448,2.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2458,6.0,4.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
100894,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
111780,3.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
37755,9.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0

## Building Models
* Naive Bayes (BernoulliNB)
* DecisionTree
* RandomForest
* MLPClassifier(NN)


In [19]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
bnb.fit(X_train, y_train)
prediction_bnb = bnb.predict_proba(X_test)

In [20]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
prediction_gnb = gnb.predict_proba(X_test)

In [21]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
prediction_mnb = mnb.predict_proba(X_test)

In [22]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
prediction_dtc = dtc.predict_proba(X_test)

In [23]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
prediction_rfc = rfc.predict_proba(X_test)

In [24]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(X_train, y_train)
prediction_mlp = mlp.predict_proba(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)



# Evaluation process



In [25]:
def calAUC(prob, labels):
    '''
    calculate the auc and return
    :param prob: the probability ndarray that predict as Positive
    :param labels: the real labels，in which 1 represent Positive，0 represent Negtive
    :return: AUC
    '''
    
    f = list(zip(prob,labels))
    
    rank = [values2 for values1,values2 in sorted(f,key = lambda x:x[0])]
    
    rankList = [i+1 for i in range(len(rank)) if rank[i] == 1]
    # get the rank
    posNum = 0
    negNum = 0
    for i in range(len(labels)):
        if(labels[i] == 1):
            posNum += 1
        else:
            negNum += 1
    # calculate the AUC
    auc = (sum(rankList) - (posNum*(posNum+1))/2)/(posNum*negNum)
    return auc

In [26]:
# use the models to predict the column that represent the probability positive
bnb_result = bnb.predict_proba(X_test)[:,1]
gnb_result = gnb.predict_proba(X_test)[:,1]
mnb_result = mnb.predict_proba(X_test)[:,1]
dtc_result = dtc.predict_proba(X_test)[:,1]
rfc_result = rfc.predict_proba(X_test)[:,1]
mlp_result = mlp.predict_proba(X_test)[:,1]

model_results = [bnb_result, gnb_result, mnb_result, dtc_result, rfc_result, mlp_result]
model_names = ['bnb', 'gnb', 'mnb', 'dtc', 'rfc', 'mlp']

In [27]:
for i, model_result in enumerate(model_results):
  print(model_names[i] + " has an AUC of: " + str(calAUC(model_result, np.array(y_test))))

bnb has an AUC of: 0.6413939140550198
gnb has an AUC of: 0.6299408575710143
mnb has an AUC of: 0.6452334749686238
dtc has an AUC of: 0.573228724539214
rfc has an AUC of: 0.789633469437323
mlp has an AUC of: 0.7180402446936208


Inbalance of data exists...

In [28]:
train_df["ACTIVE"].value_counts()

0.0    109382
1.0     11992
Name: ACTIVE, dtype: int64

cross validation

In [29]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
kf = KFold(n_splits=5, random_state=42, shuffle=False)
# rf = RandomForestClassifier(n_estimators=100)
# cross_val_score(rf, X_train, y_train, cv=kf, scoring='roc_auc')

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Use GridSearchCV to find the best parameters

In [31]:
from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier()

params = {
    'n_estimators':[50,100,200],
    'max_depth':[5,10,15,20,30],
    'random_state':[0]
}

# training times will take (3*5)*5 = 75 
grid_search = GridSearchCV(rfc, param_grid=params, cv=kf, scoring='roc_auc')

In [32]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=False),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_sco

In [42]:
grid_search.best_params_

{'max_depth': 30, 'n_estimators': 200, 'random_state': 0}

In [ ]:
sorted(grid_search.best_estimator_.feature_importances_)

In [44]:
grid_search.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=30, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [50]:
rfc_updated = RandomForestClassifier(max_depth=30, n_estimators=200)
rfc_updated.fit(X_train, y_train)
prediction_rfc_updated = rfc_updated.predict_proba(X_test)

In [51]:
print("The random forest classifier with a grid search has an AUC of: " + str(calAUC(prediction_rfc_updated[:, 1], np.array(y_test))))

The random forest classifier with a grid search has an AUC of: 0.8004881257032902


# Apply the model on the test data

In [72]:
test_df = pd.read_csv('test_smiles.csv')
test_df_copy = test_df.copy()

test_df_copy["HeavyAtomCount"] = test_df_copy["SMILES"].apply(lambda x: Chem.MolFromSmiles(x).GetNumAtoms())
test_df_copy["CalcExactMolWt"] = test_df_copy["SMILES"].apply(lambda x: d.CalcExactMolWt(Chem.MolFromSmiles(x)))
test_df_copy["fr_COO"] = test_df_copy["SMILES"].apply(lambda x: f.fr_Al_COO(Chem.MolFromSmiles(x)))

test_df_copy

RDKit WARNING: [20:04:14] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [20:04:19] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [20:04:25] WARNING: not removing hydrogen atom without neighbors


,INDEX,SMILES,HeavyAtomCount,CalcExactMolWt,fr_COO
0,121375,Cc1ccc(-c2csc(NC(=O)C3=NN(c4ccccc4)C(=O)CC3)n2...,28,390.115047,0
1,121376,O=C(Nc1ccccc1)N1CC[C@@]2(CCCN(C(=O)c3cccc(F)c3...,28,381.185255,0
2,121377,CC(=O)N1C(=O)N(C(C)=O)C2C1N(C)C(=O)N2C(C)=O,20,282.096420,0
3,121378,CCOC(=O)Cn1/c(=N/C(=O)c2ccc([N+](=O)[O-])s2)sc...,26,391.029663,0
4,121379,Cc1ccc(S(=O)(=O)N2CCC(C(=O)Nc3nnc(C45CC6CC(CC(...,34,500.191583,0
...,...,...,...,...,...
40453,161828,O=C(CSc1nnc(-c2ccncc2)o1)N1CCc2ccccc2C1,25,352.099397,0
40454,161829,N=c1sccn1CC(=O)Nc1ccc(Cl)c(S(=O)(=O)N2CCOCC2)c1,26,416.037975,0
40455,161830,CC(/C=C/c1ccc2c(c1)OCO2)=N\NC(=O)c1cccc([N+](=...,26,353.101171,0
40456,161831,CC(C)Cn1c(=O)c(C(=O)Nc2cnccn2)c(O)c2ccccc21,25,338.137890,0


In [73]:
for i in range(124):
  test_df_copy["MFbitV_"+str(i)] = 0

print("wait 5 minutes please...")
# 40458*124 MF vector
MF_x_test = test_df_copy["SMILES"].apply(lambda x: np.array(AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(x),2,nBits=124)))

# vector_lists = []
# # concat the lists
# for i in MF_x:
#   vector_lists += np.array(i).tolist()
import itertools
vector_lists_test = list(itertools.chain(*MF_x_test))

test_df_copy.iloc[:,5:] = np.array(vector_lists_test).reshape(len(test_df_copy),124)

wait 5 minutes please...


RDKit WARNING: [20:05:07] WARNING: not removing hydrogen atom without neighbors


In [76]:
test_df_data = test_df_copy.drop(columns=['INDEX','SMILES'])

In [77]:
_X_test = np.array(test_df_data)
_X_test[:,[0,1]] = scaler.fit_transform(_X_test[:,[0,1]])
_X_test[:,[0,1]] = kbd.fit_transform(_X_test[:,[0,1]])
_X_test = pd.DataFrame(_X_test, columns=test_df_data.columns, index=test_df_copy.index)
_X_test

,HeavyAtomCount,CalcExactMolWt,fr_COO,MFbitV_0,MFbitV_1,MFbitV_2,MFbitV_3,MFbitV_4,MFbitV_5,MFbitV_6,MFbitV_7,MFbitV_8,MFbitV_9,MFbitV_10,MFbitV_11,MFbitV_12,MFbitV_13,MFbitV_14,MFbitV_15,MFbitV_16,MFbitV_17,MFbitV_18,MFbitV_19,MFbitV_20,MFbitV_21,MFbitV_22,MFbitV_23,MFbitV_24,MFbitV_25,MFbitV_26,MFbitV_27,MFbitV_28,MFbitV_29,MFbitV_30,MFbitV_31,MFbitV_32,MFbitV_33,MFbitV_34,MFbitV_35,MFbitV_36,...,MFbitV_84,MFbitV_85,MFbitV_86,MFbitV_87,MFbitV_88,MFbitV_89,MFbitV_90,MFbitV_91,MFbitV_92,MFbitV_93,MFbitV_94,MFbitV_95,MFbitV_96,MFbitV_97,MFbitV_98,MFbitV_99,MFbitV_100,MFbitV_101,MFbitV_102,MFbitV_103,MFbitV_104,MFbitV_105,MFbitV_106,MFbitV_107,MFbitV_108,MFbitV_109,MFbitV_110,MFbitV_111,MFbitV_112,MFbitV_113,MFbitV_114,MFbitV_115,MFbitV_116,MFbitV_117,MFbitV_118,MFbitV_119,MFbitV_120,MFbitV_121,MFbitV_122,MFbitV_123
0,7.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,7.0,6.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
2,2.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,6.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40453,6.0,5.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
40454,6.0,8.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
40455,6.0,5.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
40456,6.0,4.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,

In [88]:
# use the model to predict the data
prediction_test_data = rfc_updated.predict(_X_test)
test_df["predicted_ACTIVE"] = prediction_test_data
test_df.to_csv("predicted_result.csv", index=False)